# **SME0142 - Álgebra Linear e Aplicações**
Docente: Cynthia de Oliveira Lage Ferreira


# **Trabalho Final**

### Aluno 1: Gabriel Natal Coutinho (12543461) - Turma das 8:10h (Turma B)
### Aluno 2: Gustavo Lelli Guirao (11918182) - Turma das 8:10h (Turma B)
### Aluno 3: Rafael Sartori Vantin (12543353) - Turma das 8:10h (Turma B)


## Reconhecimento de quádricas

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

### Introdução

As quádricas são subespaços de $R^3$ de dimensão $2$ (pois são superfícies).<br> Dado isso, a condição para um ponto $(x, y, z)$ pertencer à uma quádrica é ele satisfazer a seguinte equação:

$ax^2 + by^2 + cz^2 + dxy + exz + fyz + gx + hy + iz + j = 0$ <br>
em que ao menos $a, b, c, d, e$  ou $f \neq 0.$ <br><br>

Dessa forma, o espaço vetorial $Q(R^3)$, de todas as quádricas em $R^3$, é subespaço de $P_2(R^3)$ (espaço vetorial de todos os polinômios em $R^3$ de grau $2$), gerado pela base canônica:

$P_2(R^3) = [x^2, y^2, z^2, xy, xz, yz, x, y, z, 1]$<br><br>

Mas de forma mais precisa, o espaço $Q(R^3)$ representa o espaço de todas as raízes de um polinômio $p ∈ P_2(R^3)$ de grau exclusivamente $2$ (por esta razão que ao menos 1 dos 6 primeiros coeficientes deve ser não nulo).

### Definição das classes

Visto isso, construiremos classes para auxiliar na construção e interpretação de quádricas ao longo do trabalho:

In [ ]:
# verifica se o número é aprixmadamente 0 dada as casas de precisão
def aprox_zero(num, prec = 10):
    return np.abs(np.round(num, prec)) == 0
  
# verifica se os elementos do vetor são aproximadamente 0
def all_zeros(v, prec = 10):
    for i in v:
        if not aprox_zero(i, prec):
            return False
    return True

In [ ]:
# polinômio de R3 com seus 10 coeficientes da equação
class Polinomio_R3:
    # inicializa os coeficientes
    def __init__(self, coef):
        if len(coef) >= 10:
            self.coef = coef[:10]
        else:
            self.coef = coef + (10 - len(coef)) * [0]

    # retorna uma string da função do polinômio
    def __str__(self):
        str_vars = ['x^2', 'y^2', 'z^2', 'xy', 'xz', 'yz', 'x', 'y', 'z', '']
        str_coef = [str(np.abs(round(x, 3))) for x in self.coef]

        str_all = ''
        for i in range(len(str_coef)):
            if not aprox_zero(self.coef[i], 7):
                if len(str_all) > 0 and self.coef[i] > 0:
                    str_all += ' + '
                elif len(str_all) > 0 and self.coef[i] < 0:
                    str_all += ' - '
                elif self.coef[i] < 0:
                    str_all += '-'

                if str_coef[i] != '1':
                    str_all += str_coef[i] + str_vars[i]
                else:
                    str_all += str_vars[i]

        return str_all

    # retorna o valor da função polinomial no ponto (x, y, z) dado
    def solve(self, coord):
        [x, y, z] = coord
        [a, b, c, d, e, f, g, h, i, j] = self.coef
        res = a*x**2 + b*y**2 + c*z**2 + d*x*y + e*x*z + f*y*z + g*x + h*y + i*z + j
        return res

In [ ]:
# calcula as raízes de uma equação ax^2 + bx + c = 0 usando báskara
def raizes_eq2(a, b, c):
    # se a for 0, é uma equação de 1° grau
    if aprox_zero(a, 10):
        return (-c / b, None)

    delta = b**2 - 4 * a * c
    if aprox_zero(delta, 14): # uma raíz
        return (-b / (2 * a), None)
    elif delta < 0: # sem raíz
        return (None, None)
    else: # duas raízes
        return ((-b + np.sqrt(delta)) / (2 * a), (-b - np.sqrt(delta)) / (2 * a))

In [ ]:
# quádricas em R3, pertencentes aos polinômios em R3
class Quadrica(Polinomio_R3):
    # inicializa os coeficientes
    def __init__(self, coef):
        super().__init__(coef)

        # se os coeficientes não respeitam as condições para gerar a quádrica
        if self.coef[:6] == 6 * [0]:
            self.coef = None
            raise ValueError('6 primeiros coeficientes nulos não satisfaz a equação de uma quádrica')

    # retorna uma string da equação da quádrica
    def __str__(self):
        str_all = super().__str__()
        return str_all + ' = 0'

    # veirifica se um ponto de R3 pertence à quádrica
    def has(self, coord):
        res = super().solve(coord)
        return aprox_zero(res, 10)

    # retorna os valores de y para (x, y) pertencer à quádrica (z não considerado na equação)
    def solve_y(self, x):
        [a, b, c, d, e, f, g, h, i, j] = self.coef
        return raizes_eq2(b, d*x + h, a*x**2 + g*x + j)

    # retorna os valores de z para o ponto (x, y, z) pertençer à quádrica
    def solve_z(self, coord):
        [x, y] = coord
        [a, b, c, d, e, f, g, h, i, j] = self.coef
        return raizes_eq2(c, e*x + f*y + i, a*x**2 + b*y**2 + d*x*y + g*x + h*y + j)

    # mostra o gráfico 3d da quádrica criada
    def plot(self, lim_x, lim_y, num_points):
        df = []
        [a, b, c, d, e, f, g, h, i, j] = self.coef

        # se a equação possui todos coeficientes de z nulos, calcula y para cada x e guarda os pontos
        if all_zeros([c, e, f, i], 10):
            for x in np.linspace(lim_x[0], lim_x[1], num_points):
                for z in np.linspace(lim_y[0], lim_y[1], num_points):
                    res = self.solve_y(x)
                    if res[0] != None:
                        df.append([x, res[0], z])
                    if res[1] != None:
                        df.append([x, res[1], z])

        # calcula z para cada par (x, y) e guarda os pontos
        else:         
            for x in np.linspace(lim_x[0], lim_x[1], num_points):
                for y in np.linspace(lim_y[0], lim_y[1], num_points):
                    res = self.solve_z([x, y])
                    if res[0] != None:
                        df.append([x, y, res[0]])
                    if res[1] != None:
                        df.append([x, y, res[1]])


        # converte os dados dos pontos em um dataframe
        df = pd.DataFrame(df)
        df.columns=['x', 'y', 'z']
        
        # plot dos pontos
        fig = go.Figure(data=[go.Scatter3d(
            x=df.x,
            y=df.y,
            z=df.z,
            mode='markers',
            marker=dict(
                size=12,
                color=df.z,
                colorscale='Viridis'
            )
        )])
        
        fig.update_layout(autosize=False, width=500, height=500, margin=dict(l=0, r=0, b=0, t=0))
        fig.show()

Com isso, temos agora uma classe para visualizar e fazer análises básicas sobre qualquer quádrica que desejarmos estudar. A célula abaixo mostra as funcionalidades de escrita e plotagem da quádrica dado os 10 coeficientes (de *a* a *j*) passados.

In [ ]:
# especificações da quádrica e plot
x_range = [-15, 15]
y_range = [-15, 15]
num_points = 200

q_coef = [2, 1, 1, 2, 0, 1, 2, -3, 0, 2]

In [ ]:
q = Quadrica(q_coef)
print(q, end='\n\n')
q.plot(x_range, y_range, num_points)

2x^2 + y^2 + z^2 + 2xy + yz + 2x - 3y + 2 = 0



### Processo de rotação e translação para o reconhecimento

Analisando seu gráfico, pode-se notar rapidamente o tipo de quádrica escolhido. Contudo, analisando apenas sua equação, se torna muito complicado identificar seu tipo e qualquer propriedade sua.

Dessa forma, iremos rotacionar e transladar o espaço $R^3$ em que a quádrica está de forma que sua equação se torne muito mais simples de interpretar.

Para eliminar os termos cruzados (coeficientes *d,e,f*), faremos uma mudança de coordenadas de $(x,y,z)$ para $(x',y',z')$, saindo de uma base ortonormal canônica para uma base ortonormal formada por autovetores.

Considere a equação:

$ax^2 + by^2 + cz^2 + dxy + exz + fyz + gx + hy + iz + j = 0$

Sendo $a,b,c,d,e,f,g,h,i \in R$, sendo pelo menos $d$ ou $f$ $\neq 0$.

Esta equação tem uma versão matricial:

$
\begin{bmatrix}
x & y & z \\
\end{bmatrix} 
\begin{bmatrix}
a & d/2 & e/2 \\
d/2 & b & f/2 \\
e/2 & f/2 & c \\
\end{bmatrix} 
\begin{bmatrix}
x \\
y \\
z \\
\end{bmatrix}
+
\begin{bmatrix}
g & h & i \\
\end{bmatrix}
\begin{bmatrix}
x \\
y \\
z \\
\end{bmatrix}
+
\begin{bmatrix}
j
\end{bmatrix}
=
\begin{bmatrix}
0
\end{bmatrix}
$

Seja:

$A = 
\begin{bmatrix}
a & d/2 & e/2 \\
d/2 & b & f/2 \\
e/2 & f/2 & c \\
\end{bmatrix} $
<br><br>

Como A é uma matriz simétrica, pelo Teorema Espectral, existe uma base ortonormal β de $R^3$ formada por autovetores de A. Assim se $\lambda_1$, $\lambda_2$, $\lambda_3$ são autovalores de A (não necessariamente distintos), existem autovetores $v_1$, $v_2$, $v_3$ associados a eles, daí $β = \{v_1,v_2,v_3\}$. Daí:

$
P :=
\begin{bmatrix}
v1 & v2 & v3 \\
\end{bmatrix} 
$

$
D :=
\begin{bmatrix}
\lambda_1 & 0 & 0 \\
0 & \lambda_2 & 0 \\
0 & 0 & \lambda_3 \\
\end{bmatrix} 
$
<br><br>
Com $P^{-1} = P^T$, então:

$
A = P^{-1}DP^T \implies A = PDP^T
$

Substituindo na equação:

$
(\begin{bmatrix}
x & y & z \\
\end{bmatrix}P)D
(P^T
\begin{bmatrix}
x \\
y \\
z \\
\end{bmatrix})
+
\begin{bmatrix}
g & h & i \\
\end{bmatrix}
\begin{bmatrix}
x \\
y \\
z \\
\end{bmatrix}
+
\begin{bmatrix}
j
\end{bmatrix}
=
\begin{bmatrix}
0
\end{bmatrix}
$

$
\begin{bmatrix}
x' & y' & z' \\
\end{bmatrix}
\begin{bmatrix}
\lambda_1 & 0 & 0 \\
0 & \lambda_2 & 0 \\
0 & 0 & \lambda_3 \\
\end{bmatrix}
\begin{bmatrix}
x' \\
y' \\
z' \\
\end{bmatrix}
+
\begin{bmatrix}
g & h & i \\
\end{bmatrix}
\begin{bmatrix}
v1 & v2 & v3 \\
\end{bmatrix}
\begin{bmatrix}
x' \\
y' \\
z' \\
\end{bmatrix}
+
\begin{bmatrix}
j
\end{bmatrix}
=
\begin{bmatrix}
0
\end{bmatrix}
$

Chamando $[v]_\beta$ de $
\begin{bmatrix}
x' \\
y' \\
z' \\
\end{bmatrix}$, $v_n = (x_n, y_n, z_n)$ e fazendo manipulações obtemos:

$
λ_1x'^2 + λ_2y'^2 + λ_3z'^2 + (gx_1 + hy_1 + iz_1)x' + (gx_2 + hy_2 + iz_2)y'+ (gx_3 + hy_3 + iz_3)z'+ j = 0
$

Assim, com a mudança de base, os termos mistos foram eliminados.

In [ ]:
# rotaciona a quádrica de coeficientes dados de forma que seus termos cruzados (xy, xz e yz) se anulem
def rotate(coef):
    # matrizes dos coeficientes, originados da interpretação matricial da quádrica
    A = np.array([[coef[0], coef[3]/2, coef[4]/2], 
                  [coef[3]/2, coef[1], coef[5]/2], 
                  [coef[4]/2, coef[5]/2, coef[2]]]
    )
    B = np.array([coef[6], coef[7], coef[8]])

    # cálculo dos autovalores w e autovetores v de A
    w, v = np.linalg.eig(A)

    # matrizes dos novos coeficientes (interpretação matricial da quádrica após a transformação de rotação)
    D = np.multiply(w, np.identity(3))
    E = np.dot(B, v)

    # novos coeficientes da quádrica
    new_coef = [D[0,0], D[1,1], D[2,2], 0, 0, 0, E[0], E[1], E[2], coef[9]]
    return new_coef

Dessa forma, aplicando a função de rotação nos coeficientes da quádrica e plotando a nova equação gerada por ele, teremos:

In [ ]:
rot_coef = rotate(q.coef)

q1 = Quadrica(rot_coef)
print(q1, end='\n\n')
q1.plot(x_range, y_range, num_points)

2.662x^2 + 1.179y^2 + 0.159z^2 - 0.012x + 1.695y + 3.182z + 2 = 0



A translação da quádrica é mais simples, basta completar os quadrados como por exemplo:

$
ax^2 + bx + c = a(x + \frac{b}{2a})^2 + (c - \frac{b^2}{4a^2})
$

E então fazer uma substituição de variável:

$
x' = x + \frac{b}{2a}
$

Aplique isso para $x,y,z$ na equação da quádrica atualizando os coeficientes e o termo independente e então executamos com sucesso a translação.

In [ ]:
# calcula o desconto necessário para o coeficiente j ao reduzir um quadrado perfeito nos coeficientes
def disc_j(coef, i):
    if aprox_zero(coef[i], 10):
        return 0
    return coef[i+6]**2 / (4 * coef[i] ** 2) * coef[i]

# verifica se o coeficiente em questão poderá ser removido da equação
def rem_coef(coef, i):
    # se seu termo quadrático for zerado, então o mantêm
    if aprox_zero(coef[i-6], 10):
        return coef[i]
    return 0

# translaciona a quádrica de coeficientes dados de forma que seus termos simples (x, y, e z) se anulem
def translate(coef):
    # calcula o novo j para anular os termos
    new_j = coef[9] - disc_j(coef, 0) - disc_j(coef, 1) - disc_j(coef, 2)
    new_coef = [coef[0], coef[1], coef[2], 0, 0, 0, rem_coef(coef, 6), rem_coef(coef, 7), rem_coef(coef, 8), new_j]

    return new_coef

Aplicando então a função de translação nos coeficientes da quádrica já rotacionados e plotando a nova equação gerada por ele, teremos:

In [ ]:
final_coef = translate(rot_coef)

q2 = Quadrica(final_coef)
print(q2, end='\n\n')
q2.plot(x_range, y_range, num_points)

2.662x^2 + 1.179y^2 + 0.159z^2 - 14.5 = 0



### Reconhecimento da quádrica

Com os processos de rotacionar e transladar a quádrica dada inicialmente, pôde-se ajustá-la perpendicularmente em relação aos eixos de coordenadas e deixar seu centro na origem (0, 0, 0).

Contudo, o maior benefício que isso nos trouxe foi a simplificação de sua equação. Temos agora uma equação em que apenas os principais coeficientes para descrever a quádrica dada não são nulos. Portanto, usaremos seus valores para classificá-la.

In [ ]:
# ajusta os coeficientes para representarem apenas a quádrica no eixo principal
def move_to_principal_axis(coef):
    [a, b, c, d, e, f, g, h, i, j] = coef

    # 3 termos quadrados, não modifica nada
    if np.count_nonzero([a,b,c]) == 3:
        return coef

    # 2 termos quadrados
    if np.count_nonzero([a,b,c]) == 2:
        # caso x y2 z2 para x2 y2 z
        if a == 0:
            a = c; c = 0; i = g; g = 0
        # caso x2 y z2 para x2 y2 z
        elif b == 0:
            b = c; c = 0; i = h; h = 0;

    # 1 termo quadrado
    elif np.count_nonzero([a,b,c]) == 1:
        # caso x y2 z para x2 y z
        if b != 0:
            a = b; b = 0;  h = g; g = 0;
        # caso x y z2 para x2 y z
        elif c != 0:
            a = c; c = 0;  i = g; g = 0;

    return [a, b, c, d, e, f, g, h, i, j]

# classifica a quádrica passada de acordo com seus coeficientes
def classify_q(coef):
    # arredonda e ajusta os coeficientes antes da classificação
    coef = np.around(coef, decimals = 5)
    coef = move_to_principal_axis(coef)
    [a, b, c, d, e, f, g, h, i, j] = coef

    # quádricas de 3 termos quadrados
    if np.count_nonzero([a,b,c]) == 3:
        # elipsoides
        if a * b > 0 and a * c > 0 and a * j < 0:
            if a == b and a == c:
                return 'Esfera'
            else:
                return 'Elipsoide'

        # hiperboloides
        elif ((a * b) * (a * c) < 0 or a * b * c < 0):
            if j == 0:
                return 'Cone'
            elif a * b * c * j < 0:
                return 'Hiperboloide de 2 folhas'
            else:
                return 'Hiperboloide de 1 folha'

    # quádricas de 2 termos quadrados
    elif np.count_nonzero([a,b,c]) == 2:
        # paraboloides
        if i != 0:
            if a * b > 0:
                return 'Paraboloide elíptico'
            elif a * b < 0:
                return 'Paraboloide hiperbólico'

        # cilindros (ou cônicas de R2 no espaço de R3)
        else:
            if a * b > 0 and a * j < 0:
                if a == b:
                    return 'Cilindro circular'
                else:
                    return 'Cilindro elíptico'
            elif a * b < 0 and j != 0:
                return 'Cilindro hiperbólico'

    # quádricas de 1 termo quadrado
    elif np.count_nonzero([a,b,c]) == 1:
        if h != 0 or i != 0:
            return 'Cilindro parabólico'

    # se não passou em nenhuma classificação
    return 'Quádrica degenerada'

Classificando então a quádrica simplificada:

In [ ]:
classify_q(final_coef)

'Elipsoide'

### Resumo

Em resumo ao código escrito, podemos juntar todas as funções em uma só para fazer diretamente o que queremos: reconhecer a quádrica com seus coeficientes passados.

In [ ]:
# reconhece a quádrica de coeficientes passados
def rec_quadrica(coef):
    q = Quadrica(coef)
    print(f'Quádrica (plotada):\n{q}\n')

    simpl_coef = rotate(coef)
    q_simpl = Quadrica(simpl_coef)
    print(f'Versão desrotacionada:\n{q_simpl}\n')

    simpl_coef = translate(simpl_coef)
    q_simpl = Quadrica(simpl_coef)
    print(f'Versão destransladada:\n{q_simpl}\n')
    print(f'Tipo: {classify_q(simpl_coef)}\n')

    q.plot(x_range, y_range, num_points)

utilizando outra quádrica para exemplificar o processo resumido:

In [ ]:
q_coef = [0, 1, 1, 2, 0, 1, 2, -3, 0, 2]
rec_quadrica(q_coef)

Quádrica (plotada):
y^2 + z^2 + 2xy + yz + 2x - 3y + 2 = 0

Versão desrotacionada:
-0.662x^2 + 1.841y^2 + 0.821z^2 + 3.279x - 1.49y + 0.176z + 2 = 0

Versão destransladada:
-0.662x^2 + 1.841y^2 + 0.821z^2 + 5.75 = 0

Tipo: Hiperboloide de 2 folhas



### Observações

Neste trabalho, não nos preocupamos em fazer o reconhecimento específico de quádricas degeneradas (aquelas que podem representar uma reta, um plano, um par de planos paralelos, um par de planos transversais, um ponto ou o conjunto vazio). O principal motivo disso são os diversos problemas da função de plot com esse tipo de quádricas.

Alguns coeficientes de quádricas utilizados para testar o trabalho:

[-9, 36, 4, 0, 0, 1, 0, -216, -16, 304] => hiperboloide 1 folha <br>
[0, 1, 1, 2, 0, 1, 2, -3, 0, 2] => hiperboloide 2 folhas <br>
[2, 1, 1, 2, 0, 1, 2, -3, 0, 2] => elipsoide <br>
[-9, 16, 4, 0, 0, 1, 0, -15.97, 0, 4.0006415686274] => cone <br>
[2, 3, 0, 4, 0, 0, 2, -3, 3, 4] => paraboloide elíptico <br>
[0, 0.02, 0, 4, 0, 3, 2, -3, 0, 2] => paraboloide hiperbólico <br>
[0, 3, 2, 0, 0, 3, 0, 7, -4, -12] => cilindro elíptico <br>
[0, 0.2, -3, 0, 0, 4, 0, 7, -4, -12] => cilindro hiperbólico <br>
[0, 0.2, 0, 0, 0, 0, 0, 7, -4, 2] => cilindro parabólico <br>